**Importing Libraries**

Untuk menggunakan fungsi Python, kita perlu mengimpornya terlebih dahulu. Kami mengimpor perpustakaan ini:

1. Pandas: Library untuk menangani data
2. Numpy: Library untuk menangani komputasi numerik dan array
3. Matplotlib: Library untuk plotting dan visualisasi
4. Seaborn: Library untuk visualisasi, versi matplotlib yang lebih keren dan lebih ramah pengguna

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

**Load Data**

Pertama dan terpenting, mari kita mulai petualangan kita dengan memuat data kita. Mari gunakan dataset berikut yang berisi stat review buku dari goodreads.

Sumber data: https://www.kaggle.com/jealousleopard/goodreadsbooks

In [ ]:
df_book=pd.read_csv('../input/goodreadsbooks/books.csv', error_bad_lines = False)
df_book.columns = df_book.columns.str.strip()

Langkah pertama adalah membiasakan diri dengan data, mulai dengan melihat jumlah baris dan kolom, mengintip tampilan datanya, dan apa saja definisi dan tipe untuk setiap kolom.

In [ ]:
print(df_book.shape)

In [ ]:
print(df_book.isna().sum())

Ya, untungnya kami tidak memiliki data nol (kosong) pada kumpulan data ini, namun ini bukan kasus umum untuk kumpulan data lain.
Mari kita visualisasikan data dengan cara yang paling sederhana, dengan menampilkan beberapa di antaranya dalam bentuk tabel

In [ ]:
df_book.head(5)

**Data Summaries**

In [ ]:
df_book.dtypes

Deskripsi Kolom:

1. bookID Berisi ID unik untuk setiap buku / seri
2. judul berisi judul-judul buku
3. penulis berisi penulis buku tertentu
4. average_rating nilai rata-rata buku, seperti yang diputuskan oleh pengguna
5. ISBN Nomor ISBN (10), memberi tahu informasi tentang sebuah buku - seperti edisi dan penerbit
6. ISBN 13 Format baru untuk ISBN, diterapkan pada tahun 2007. 13 digit
7. language_code Memberi tahu bahasa untuk buku-buku
8. Num_pages Berisi jumlah halaman buku
9. Ratings_count Berisi jumlah peringkat yang diberikan untuk buku tersebut
10. text E5E5E5_count Memiliki jumlah ulasan yang ditinggalkan oleh pengguna

Tipe data ini biasanya muncul dengan Python:

1. Bilangan bulat
2. Float (Desimal)
3. Objek: Biasanya berisi string
4. Stempel waktu

**Data Preprocess**

Panda memberikan fleksibilitas dalam menangani pemrosesan awal data. Dari slicing data, transforming column types, aggregating, dan masih banyak hal lain yang bisa kita lakukan untuk mempersiapkan data kita dengan baik.

**Selecting Subset**

In [ ]:
df_subset=df_book[df_book['authors'].isin(['J.K. Rowling','Stephen King'])]

In [ ]:
df_subset.head(5)

In [ ]:
df_subset.groupby('authors').count()

Mari gunakan Regex untuk mendapatkan buku dari J.K Rowling terlepas dari itu ditulis sendiri atau digabungkan oleh penulis lain.

In [ ]:
df_subset=df_book[df_book['authors'].str.contains('.*J.K. Rowling.*', regex=True)==True]

In [ ]:
df_subset.groupby('authors').count()

**Transforming Values**

Mari kita coba membuat tiga kolom baru yang berbeda, yaitu:

1. Publication year
2. Grouping by number of pages
3. First Author

In [ ]:
# Dapatkan tahun penerbitan buku dengan mengambil 4 karakter terakhir
# Slicing data sekali lagi, straightforward
df_book['publication_year']=df_book['publication_date'].str[-4:]

In [ ]:
# Buat label baru berdasarkan jumlah halaman buku
# Menggunakan CASE-SELECT logic
df_book['num_pages_grp']= np.select(
    [
        df_book['num_pages'].between(0, 249, inclusive=True), 
        df_book['num_pages'].between(250, 499, inclusive=True),
        df_book['num_pages'].between(500, 749, inclusive=True),
        df_book['num_pages'].between(750, 999, inclusive=True),
        df_book['num_pages']>=1000    
    ], 
    [
        '<250', 
        '250-499',
        '500-749',
        '750-999',
        '>=1000'
    ], 
    default='Unknown'
)

In [ ]:
# Gunakan regex untuk mengekstrak penulis sebelum '/' pertama atau mengembalikan nilai kolom mentah jika tidak ada karakter '/'
# RegEx (Regular Expression) adalah cara umum untuk menentukan string berdasarkan pola, akan selalu berguna tetapi sulit untuk dikuasai
df_book['first_author']=df_book['authors'].str.extract(r'(.*?)\/')
df_book['first_author']=df_book['first_author'].fillna(df_book['authors'])

In [ ]:
df_book.head(5)

Type Markdown and LaTeX:  𝛼2

In [ ]:
df_book.groupby('num_pages_grp').count()

**Aggregation & Sorting**

Aggregating and sorting di Pandas itu intutif dan selalu berguna. Terkadang wawasan yang berharga dapat datang hanya dengan melihat data top-n (misalnya aturan Pareto), atau hanya dengan melihat properti top-n.

Mari kita dapatkan 10 penulis teratas.

In [ ]:
# Group by column 'authors'
# Kita dapat menerapkan fungsi agregasi yang berbeda untuk setiap kolom
most_books = df_book.groupby('authors').agg({'title':'count','average_rating':np.mean,'text_reviews_count':'sum'})
# Menyortir di Pandas juga mudah
# Dapatkan 10 data teratas
most_books = most_books.sort_values('title', ascending=False).head(10)

In [ ]:
most_books

Kita dapat melihat bahwa meskipun penulis memiliki jumlah judul yang sama, jumlah ulasan teks mereka mungkin berbeda secara signifikan. Mungkinkah ini karena popularitas?


**Data Visualization**

1D Plotting

1D Plotting hanya menggunakan nilai dari satu kolom atau dimensi. Biasanya plot-plot ini berguna untuk merepresentasikan distribusi data.

Histogram

In [ ]:
plt.figure(figsize=(20,20))
g =sns.displot(df_book, x="language_code")
g.fig.set_figwidth(20)

KDE

In [ ]:
g=sns.displot(df_book, x="average_rating", kind="kde")
g.fig.set_figwidth(20)

Barplot

In [ ]:
#sns.set_context('talk')
most_books = df_book.groupby('authors')['title'].count().reset_index().sort_values('title', ascending=False).head(10).set_index('authors');
plt.figure(figsize=(15,10));
ax = sns.barplot(most_books['title'], most_books.index, palette='icefire_r');
ax.set_title("Top 10 authors with most books");
ax.set_xlabel("Total number of books");
for i in ax.patches:
    ax.text(i.get_width()+.3, i.get_y()+0.5, str(round(i.get_width())), fontsize = 10, color = 'k');

**2D Plotting**

Stack bar

In [ ]:
g=sns.displot(df_book[df_book['publication_year'].between('2003','2007')], x="publication_year", hue="num_pages_grp", multiple="stack")
g.fig.set_figwidth(15)
g.fig.set_figheight(8)

Scatter Plot

In [ ]:
df_subset = df_book[df_book['authors'].str.contains('.*Agatha Christie.*', regex=True)==True]
df_subset = df_subset[df_subset['publication_year'].between('1990','2010')]
df_subset['publication_year'] = df_subset['publication_year'].astype(int)
fig, ax = plt.subplots(figsize=(15,10))
sns.regplot(ax=ax,data=df_subset, x="publication_year", y="average_rating")
ax.set_xticks(list(range(1990,2011,4)));

Scatter plot berguna untuk melihat korelasi atau trend antara sumbu X dan sumbu Y. Selanjutnya dilakukan pencocokan kurva atau regresi antara variabel X dan Y. Sebagai contoh, kita dapat melihat bahwa novel Agata Christie mendapatkan nilai rata-rata yang lebih rendah dari waktu ke waktu

Heatmap

In [ ]:
most_books_publisher = df_book.groupby('publisher')['title'].count().reset_index().sort_values('title', ascending=False).head(10);
df_subset = df_book[df_book['publisher'].isin(most_books_publisher.reset_index()['publisher'].tolist())]
df_subset = df_subset[df_subset['publication_year'].isin(['2003','2004','2005','2006','2007'])]
df_subset = df_subset.groupby(["publisher","publication_year"]).agg({"average_rating":np.mean}).reset_index()
df_subset = df_subset.pivot("publisher", "publication_year", "average_rating")
df_subset

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
g = sns.heatmap(ax=ax,data=df_subset)

Violin Plot

Violin Plot berguna untuk melihat penyebaran data pada suatu dimensi di berbagai kategori. Misalnya, mari kita lihat penyebaran ulasan peringkat rata-rata di seluruh penulis top.

In [ ]:
most_books = df_book.groupby('authors')['title'].count().reset_index().sort_values('title', ascending=False).head(10).set_index('authors');
df_most_books = df_book[df_book['authors'].isin(most_books.index.tolist())]

In [ ]:
plt.figure(figsize=(15,10));
ax = sns.violinplot(x="authors", y="average_rating",
                    data=df_most_books, palette="muted")
ax.set_xticklabels(ax.get_xticklabels(),rotation=30);

Variation

Wordcloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
text=df_most_books['title'].tolist()
def plot_word_cloud(df_cloud):
    stopwords = set(STOPWORDS)
    comment_words = ''
    for val in df_cloud['title']:
      
        # typecaste each val to string
        val = str(val)

        # split the value
        tokens = val.split()

        # Converts each token into lowercase
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower()

        comment_words += " ".join(tokens)+" "

        wordcloud = WordCloud(width = 800, height = 800,
                    background_color ='white',
                    stopwords = stopwords,
                    min_font_size = 10).generate(comment_words)

    #plot the WordCloud image                       
    plt.figure(figsize = (8, 8), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)

    plt.show()

In [ ]:
# iterate through the csv file
df_cloud=df_most_books[df_most_books['authors']=='Stephen King']
plot_word_cloud(df_cloud)

In [ ]:
df_cloud=df_most_books[df_most_books['authors']=='Agatha Christie']
plot_word_cloud(df_cloud)